In [1]:
pip install codecarbon

  Using cached codecarbon-2.4.2-py3-none-any.whl.metadata (8.4 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pynvml-11.5.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached rapidfuzz-3.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached codecarbon-2.4.2-py3-none-any.whl (494 kB)
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
Using cached pynvml-11.5.0-py3-none-any.whl (53 kB)
Using cached rapidfuzz-3.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow_datasets

In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import pandas as pd
from codecarbon import OfflineEmissionsTracker

2024-05-22 14:43:14.256537: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 14:43:31.012216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load li

In [5]:
tf_version = tf.__version__
print(f"TensorFlow version: {tf_version}")

keras_version = tf.keras.__version__
print(f"Keras version: {keras_version}")
print(f"Keras version: {hub.__version__}")
print(f"Keras version: {tfds.__version__}")
print(f"Keras version: {pd.__version__}")

TensorFlow version: 2.12.0
Keras version: 2.12.0
Keras version: 0.16.1
Keras version: 4.9.4
Keras version: 2.0.3


In [3]:
training_set=tf.data.Dataset.load('gs://plant_buddy/oxford_dataset/training_set')
validation_set=tf.data.Dataset.load('gs://plant_buddy/oxford_dataset/validation_set')
test_set=tf.data.Dataset.load('gs://plant_buddy/oxford_dataset/test_set')

training_class_counts = {}
validation_class_counts = {}
test_class_counts = {}

for images, labels in training_set:
    for label in labels.numpy():
        label = label.item()  
        if label in training_class_counts:
            training_class_counts[label] += 1
        else:
            training_class_counts[label] = 1


for images, labels in validation_set:
    for label in labels.numpy():
        label = label.item()  
        if label in validation_class_counts:
            validation_class_counts[label] += 1
        else:
            validation_class_counts[label] = 1


for images, labels in test_set:
    for label in labels.numpy():
        label = label.item() 
        if label in test_class_counts:
            test_class_counts[label] += 1
        else:
            test_class_counts[label] = 1

In [4]:
def format_image(image, label):
    image = image * 255.0
    return image, label

training_set = training_set.map(format_image)
validation_set = validation_set.map(format_image)
test_set = test_set.map(format_image)

In [5]:
for image, label in training_set.take(1): 
    print("Sample image pixel values range:")
    print("Min pixel value:", tf.reduce_min(image).numpy())
    print("Max pixel value:", tf.reduce_max(image).numpy())
    print("Sample pixel values:", image.numpy()[0, :5, :5, 0])

Sample image pixel values range:
Min pixel value: 0.0
Max pixel value: 255.0
Sample pixel values: [[77.64955  73.77455  68.298134 64.15179  61.692383]
 [74.41741  69.6942   63.959816 57.30357  55.601704]
 [73.345985 68.1875   60.65625  58.       56.390625]
 [73.640625 67.875    60.65625  56.91211  54.950195]
 [71.72098  66.75349  60.65625  56.79534  52.93527 ]]


In [6]:

print("Training set labels:")
for label, count in training_class_counts.items():
    print("{}".format(label))

print("Training set:")
for label, count in training_class_counts.items():
    print("{}".format(count))

print("\nValidation set labels:")
for label, count in validation_class_counts.items():
    print("{}".format(label))

print("\nValidation set:")
for label, count in validation_class_counts.items():
    print("{}".format(count))

print("\nTest set labels:")
for label, count in test_class_counts.items():
    print("{}".format(label))

print("\nTest set:")
for label, count in test_class_counts.items():
    print("{}".format(count))

Training set labels:
25
100
66
17
72
95
43
26
93
88
23
11
90
28
59
64
68
55
69
81
78
70
6
14
12
19
27
3
76
22
56
75
42
74
5
94
7
33
38
73
34
35
18
80
29
97
84
60
82
63
2
86
65
15
87
20
47
50
4
46
45
89
21
0
36
52
49
61
57
24
16
99
85
54
48
71
40
79
83
1
96
92
101
10
77
91
32
41
98
8
62
51
53
67
31
58
39
30
13
37
44
9
Training set:
27
46
30
62
166
70
77
26
143
158
31
73
60
61
90
86
41
86
46
94
27
64
27
36
35
43
49
44
217
74
53
89
116
101
30
107
69
29
29
151
33
57
34
137
67
68
46
32
106
38
29
50
49
31
131
28
57
228
53
54
167
64
46
28
89
73
76
45
97
29
66
33
46
54
37
76
106
88
70
47
49
32
38
70
118
50
33
44
51
33
41
68
48
41
35
52
51
36
32
42
28
32

Validation set labels:
44
8
65
93
92
5
40
85
63
51
46
39
38
21
27
47
99
75
101
37
96
69
11
72
48
74
14
55
57
79
42
76
41
36
54
32
71
94
1
91
53
80
70
50
56
43
34
26
31
3
2
60
81
16
12
78
13
23
62
82
18
68
17
29
49
4
24
59
77
90
100
15
19
6
45
33
35
95
30
84
67
83
97
87
58
52
20
9
28
98
0
22
61
86
88
10
89
64
7
73
66
25

Validation set:
9
9
6
8

In [5]:

num_training_examples = 0
num_validation_examples = 0
num_test_examples = 0

for example in training_set:
  num_training_examples += 1

for example in validation_set:
  num_validation_examples += 1

for example in test_set:
  num_test_examples += 1

print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {}'.format(num_validation_examples))
print('Total Number of Test Images: {} \n'.format(num_test_examples))

num_classes = 102
print('Total Number of Classes: {}'.format(num_classes))


# In[4]:


num_training_batches = tf.data.experimental.cardinality(training_set).numpy()
num_validation_batches = tf.data.experimental.cardinality(validation_set).numpy()
num_test_batches = tf.data.experimental.cardinality(test_set).numpy()

print("Number of batches in the training set:", num_training_batches)
print("Number of batches in the validation set:", num_validation_batches)
print("Number of batches in the test set:", num_test_batches)



Total Number of Training Images: 205
Total Number of Validation Images: 26
Total Number of Test Images: 26 

Total Number of Classes: 102
Number of batches in the training set: 205
Number of batches in the validation set: 26
Number of batches in the test set: 26


In [7]:
input_shape = (224, 224, 3)
IMAGE_RES = 224
BATCH_SIZE = 32

In [7]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
])


from tensorflow.keras import layers

URL = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES, 3))

feature_extractor.trainable = False

model = tf.keras.Sequential([
  data_augmentation,
  feature_extractor,
  layers.Dense(num_classes, activation='softmax')
])


model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

In [8]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.4),
])


from tensorflow.keras import layers

# URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
# feature_extractor = hub.KerasLayer(URL,
#                                    input_shape=(IMAGE_RES, IMAGE_RES, 3))

# base_model = tf.keras.applications.ResNet50V2(input_shape=(IMAGE_RES, IMAGE_RES, 3),
#                                                include_top=False,
#                                                weights='imagenet')
# base_model = tf.keras.applications.MobileNetV2(input_shape=(IMAGE_RES, IMAGE_RES, 3),
#                                                include_top=False,
#                                                weights='imagenet')

base_model = tf.keras.applications.EfficientNetB0(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

In [9]:
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(102, activation='softmax')
dropout_rate = 0.2

inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = layers.Dropout(dropout_rate)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 102)               130662    
                                                             

In [17]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name, layer.trainable)

0 input_1 False
1 rescaling False
2 normalization False
3 rescaling_1 False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeeze False
39

In [41]:
model = tf.keras.models.load_model('gs://plant_buddy/models/models/rest_apr_after_fine_pr', compile=False)

# # model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy']) 
base_learning_rate = 0.00001
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate), metrics=['accuracy']) 
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0     

In [11]:
base_learning_rate=0.001

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

In [16]:
base_model.trainable = True

print("Number of layers in the base model: ", len(base_model.layers))

fine_tune_at = 222

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False


Number of layers in the base model:  238


In [21]:
base_learning_rate=0.0001
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])


In [22]:
tracker = OfflineEmissionsTracker(country_iso_code="USA", measure_power_secs=600, output_dir="gs://plant_buddy/energy", output_file="emissions_effnet_after_fine_tf1.csv")
tracker.start()

history = model.fit(training_set,
        batch_size = 32,
        epochs=20,
        validation_data=validation_set)

emissions: float = tracker.stop()
print(emissions)

model.save('gs://plant_buddy/models/models/effnet_after_fine_tf1')
pd.DataFrame.from_dict(history.history).to_csv('gs://plant_buddy/histories/history_effnet_after_fine_tf1.csv',index=False)

[codecarbon INFO @ 19:00:18] offline tracker init
[codecarbon WARNING @ 19:00:18] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 19:00:18] [setup] RAM Tracking...
[codecarbon INFO @ 19:00:18] [setup] GPU Tracking...
[codecarbon INFO @ 19:00:18] No GPU found.
[codecarbon INFO @ 19:00:18] [setup] CPU Tracking...
[codecarbon WARNING @ 19:00:18] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 19:00:19] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 19:00:19] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 19:00:19] >>> Tracker's metadata:
[codecarbon INFO @ 19:00:19]   Platform system: Linux-5.10.0-28-cloud-amd64-x86_64-with-glibc2.35
[codecarbon INFO @ 19:00:19]   Python version: 3.10.13
[codecarbon INFO @ 19:00:19]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 19:00:19]   Available RAM : 29.390 GB
[codecarbon I

Epoch 1/20
205/205 [==============================] - 147s 673ms/step - loss: 0.0023 - accuracy: 0.9995 - val_loss: 0.2077 - val_accuracy: 0.9634
Epoch 2/20
205/205 [==============================] - 140s 680ms/step - loss: 0.0016 - accuracy: 0.9997 - val_loss: 0.2030 - val_accuracy: 0.9621
Epoch 3/20
205/205 [==============================] - 139s 675ms/step - loss: 0.0017 - accuracy: 0.9997 - val_loss: 0.2028 - val_accuracy: 0.9646
Epoch 4/20
205/205 [==============================] - 159s 775ms/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 0.2015 - val_accuracy: 0.9646
Epoch 5/20
  9/205 [>.............................] - ETA: 3:05 - loss: 0.0016 - accuracy: 1.0000  

[codecarbon INFO @ 19:10:19] Energy consumed for RAM : 0.001837 kWh. RAM Power : 11.021209716796875 W
[codecarbon INFO @ 19:10:19] Energy consumed for all CPUs : 0.007083 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:10:19] 0.008920 kWh of electricity used since the beginning.


205/205 [==============================] - 155s 751ms/step - loss: 0.0012 - accuracy: 0.9995 - val_loss: 0.2022 - val_accuracy: 0.9634
Epoch 6/20
205/205 [==============================] - 136s 662ms/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.2032 - val_accuracy: 0.9621
Epoch 7/20
205/205 [==============================] - 137s 664ms/step - loss: 6.9620e-04 - accuracy: 0.9998 - val_loss: 0.2040 - val_accuracy: 0.9658
Epoch 8/20
205/205 [==============================] - 136s 662ms/step - loss: 7.9105e-04 - accuracy: 0.9998 - val_loss: 0.2035 - val_accuracy: 0.9634
Epoch 9/20
 75/205 [=========>....................] - ETA: 1:17 - loss: 9.8950e-04 - accuracy: 0.9996

[codecarbon INFO @ 19:20:19] Energy consumed for RAM : 0.003674 kWh. RAM Power : 11.021209716796875 W
[codecarbon INFO @ 19:20:19] Energy consumed for all CPUs : 0.014167 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:20:19] 0.017840 kWh of electricity used since the beginning.


...

0.015233291179241456


2024-05-22 19:46:44.777447: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'random_flip_input' with dtype float and shape [?,224,224,3]
	 [[{{node random_flip_input}}]]
2024-05-22 19:46:44.878757: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,224,224,3]
	 [[{{node inputs}}]]
2024-05-22 19:46:44.890597: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'random_flip_input' with dtype float and shape [

INFO:tensorflow:Assets written to: gs://plant_buddy/models/models/effnet_after_fine_tf1/assets


INFO:tensorflow:Assets written to: gs://plant_buddy/models/models/effnet_after_fine_tf1/assets


TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

In [36]:
model.save('gs://plant_buddy/models/models/effnet_after_fine_tf11')
pd.DataFrame.from_dict(history.history).to_csv('gs://plant_buddy/histories/history_effnet_after_fine_tf1.csv',index=False)

2024-05-22 20:28:40.412168: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'random_flip_input' with dtype float and shape [?,224,224,3]
	 [[{{node random_flip_input}}]]
2024-05-22 20:28:40.530444: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,224,224,3]
	 [[{{node inputs}}]]
2024-05-22 20:28:40.544024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'random_flip_input' with dtype float and shape [

INFO:tensorflow:Assets written to: gs://plant_buddy/models/models/effnet_after_fine_tf11/assets


INFO:tensorflow:Assets written to: gs://plant_buddy/models/models/effnet_after_fine_tf11/assets


TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

In [ ]:
base_model.trainable = True

print("Number of layers in the base model: ", len(base_model.layers))

fine_tune_at = 177

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

for i, layer in enumerate(base_model.layers):
   print(i, layer.name, layer.trainable)


In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

tracker = OfflineEmissionsTracker(country_iso_code="USA", measure_power_secs=600, output_dir="gs://plant-buddy3/energy", output_file="emissions_rest_apr_after_fine.csv")
tracker.start()

history = model.fit(training_set,
        batch_size = 32,
        epochs=35,
        validation_data=validation_set)

emissions: float = tracker.stop()
print(emissions)

model.save('gs://plant-buddy3/models/rest_apr_after_fine')
pd.DataFrame.from_dict(history.history).to_csv('gs://plant-buddy3/history/history_rest_apr_after_fine.csv',index=False)

In [26]:
tracker = OfflineEmissionsTracker(country_iso_code="USA", measure_power_secs=60, output_dir="gs://plant_buddy/energy", output_file="emissions_effnet_after_fine_tf1_eval.csv")
tracker.start()

loss, accuracy = model.evaluate(test_set)
print('Test accuracy :', accuracy)
print('Test loss :', loss)

emissions: float = tracker.stop()
print(emissions)

[codecarbon INFO @ 20:13:17] offline tracker init
[codecarbon WARNING @ 20:13:17] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 20:13:17] [setup] RAM Tracking...
[codecarbon INFO @ 20:13:17] [setup] GPU Tracking...
[codecarbon INFO @ 20:13:17] No GPU found.
[codecarbon INFO @ 20:13:17] [setup] CPU Tracking...
[codecarbon WARNING @ 20:13:17] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:13:19] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:13:19] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 20:13:19] >>> Tracker's metadata:
[codecarbon INFO @ 20:13:19]   Platform system: Linux-5.10.0-28-cloud-amd64-x86_64-with-glibc2.35
[codecarbon INFO @ 20:13:19]   Python version: 3.10.13
[codecarbon INFO @ 20:13:19]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 20:13:19]   Available RAM : 29.390 GB
[codecarbon I

26/26 [==============================] - 14s 512ms/step - loss: 0.0679 - accuracy: 0.9829


[codecarbon INFO @ 20:13:33] Energy consumed for RAM : 0.000043 kWh. RAM Power : 11.021209716796875 W
[codecarbon INFO @ 20:13:33] Energy consumed for all CPUs : 0.000165 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:13:33] 0.000208 kWh of electricity used since the beginning.


Test accuracy : 0.9828850626945496
Test loss : 0.06791478395462036
7.647760525562513e-05


In [24]:
# converter = tf.lite.TFLiteConverter.from_saved_model('effnet2_apr')
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_quant_model = converter.convert()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('effnet_after_fine_tf1.tflite', 'wb') as f:
  f.write(tflite_model)

2024-05-22 20:03:47.011624: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'random_flip_input' with dtype float and shape [?,224,224,3]
	 [[{{node random_flip_input}}]]
2024-05-22 20:03:47.110567: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,224,224,3]
	 [[{{node inputs}}]]
2024-05-22 20:03:47.122630: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'random_flip_input' with dtype float and shape [

INFO:tensorflow:Assets written to: /tmp/tmp_btl_5z1/assets


INFO:tensorflow:Assets written to: /tmp/tmp_btl_5z1/assets
2024-05-22 20:05:00.341942: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-05-22 20:05:00.342017: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-05-22 20:05:00.390248: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp_btl_5z1
2024-05-22 20:05:00.457875: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-05-22 20:05:00.457939: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp_btl_5z1
2024-05-22 20:05:00.670401: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2024-05-22 20:05:00.725585: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-05-22 20:05:01.615960: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio